In [1]:
import os 
import sys
import numpy as np
from skimage.viewer import ImageViewer
import re
import glob
import collections 
from sklearn.cluster import KMeans

ros_path = '/opt/ros/kinetic/lib/python2.7/dist-packages'
if ros_path in sys.path:
    sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2


/home/pns/anaconda3/lib/python3.6/site-packages/skimage/viewer/utils/core.py:10: UserWarning: Recommended matplotlib backend is `Agg` for full skimage.viewer functionality.
  warn("Recommended matplotlib backend is `Agg` for full "


# Make Video Footage with circles
## Manually draw circles


In [2]:
path = '.'
files= [file for file in os.listdir(path) if file.endswith('.png')]
def byFrame(img):
    return  int(re.search(r'\d+', img[18:22]).group())
files.sort(key = byFrame)


In [3]:
#All circle Locations
circ_locs = [(672,358),(383,390),(412,555),(460,718),(660,696),(865,710),(890,545),(935,375)]
#(675,532) for frame 192
#(673,528) for frame 328
#(658,520) for frame 444
#(652,523) for frame 555
#(650,525) for frame 753
#(656,528) for frame 866
#(678, 539) for frame 985
avg_center = (663,528)

In [4]:
def label_circles(locs, img_name):
    img = cv2.imread(img_name,1)
    #same color circles
    for loc in locs:
        cv2.circle(img, loc, 63,(0,0,255),5)
    #center circle different color 
    cv2.circle(img, avg_center, 63, (255,0,0),5)
#    #Outer Circle 
#     cv2.circle(img, avg_center, 106, (255,0,0),2)
    return img 
def label_images(locs, img_files):
    labeled_imgs = collections.deque()
    for img_name in img_files:
        new_img = label_circles(locs, img_name)
        labeled_imgs.append(new_img)
    return labeled_imgs 

In [5]:
# labeled = label_images(circ_locs, files)

In [6]:
# height, width, layers = labeled[0].shape

# fps = 60
# video = cv2.VideoWriter('labeled_movement.avi', cv2.VideoWriter_fourcc(*'XVID'), fps, (width,height))
# for i in range(len(labeled)):    
#     video.write(labeled[i])
# cv2.destroyAllWindows()
# video.release()

# Hough Transform 

In [5]:
def find_circles(img_name):
    start_param = 30
    max_param = 50
    img = cv2.imread(img_name, 1)
    cimg = cv2.medianBlur(img,5)
    gray = cv2.cvtColor(cimg, cv2.COLOR_BGR2GRAY)
    circles = cv2.HoughCircles(gray,cv2.HOUGH_GRADIENT,1,10,param1=50,param2=start_param,minRadius=65,maxRadius=80)
    while ((circles is not None) and (circles.shape[1] > 1)):
        start_param = (start_param +max_param)/2
        circles = cv2.HoughCircles(gray,cv2.HOUGH_GRADIENT,1,10,param1=50,param2=start_param,minRadius=65,maxRadius=80)
    circ_tup = (-100, -100, -10)
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0,:]:
            # draw the outer circle
            cv2.circle(img,(i[0],i[1]),i[2],(0,255,0),2)
            # draw the center of the circle
            cv2.circle(img,(i[0],i[1]),2,(0,0,255),3)
            circ_tup = (i[0],i[1],i[2])
    return img, circ_tup
def circles_all(img_array):
    labeled = collections.deque()
    for img in img_array:
        new_img,circ_tup = find_circles(img)
        labeled.append(new_img)
    return labeled

In [6]:
def find_center_radius(img_array,num_clusters):
    circ_list = []
    for img in img_array:
        new_img,circ_tup = find_circles(img)
#         cv2.imwrite("circle" + str(byFrame(img))+".png", new_img)
        circ_list.append(circ_tup)
    circ_mat = np.array(circ_list)
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(circ_mat[:,:2])
    return circ_mat, kmeans
def calc_center_radius(circ_mat, kmeans, num_clusters):
    avg_radius = []
    for j in range(10):
        indices = np.where(kmeans.labels_ == j)
        radius = circ_m[indices,2]
        avg_radius.append(np.average(radius))
    return np.hstack((np.rint(kmeans.cluster_centers_),np.rint(avg_radius)[:,np.newaxis]))
def label_hough_circles(circles, img_name):
    img = cv2.imread(img_name,1)
    for idx in range(len(circles)):
        c = circles[idx]
        if (c > 0).all():
            if idx ==0:
                cv2.circle(img, (int(c[0]),int(c[1])), int(c[2]),(255,0,0),5)
            else:
                cv2.circle(img, (int(c[0]),int(c[1])), int(c[2]),(0,0,255),5)
    return img 
def label_hough_images(circles, img_files):
    labeled_imgs = collections.deque()
    for img_name in img_files:
        new_img = label_hough_circles(circles, img_name)
        labeled_imgs.append(new_img)
    return labeled_imgs 


In [7]:
#10 clusters because we move to 9 positions and 1 extra cluster to account for frames where no circles found.
circ_m, kmeans = find_center_radius(files,10)

In [8]:
circ_ = calc_center_radius(circ_m, kmeans, 10)
circ_

array([[ 652.,  533.,   74.],
       [-100., -100.,  -10.],
       [ 407.,  567.,   69.],
       [ 925.,  378.,   70.],
       [ 872.,  724.,   71.],
       [ 452.,  732.,   70.],
       [ 652.,  365.,   71.],
       [ 377.,  400.,   67.],
       [ 659.,  709.,   73.],
       [ 901.,  550.,   72.]])

# Auto-Draw Circles

In [9]:
#first image to half way point of images
labeled_auto = label_hough_images(circ_, files[0:1186])

In [31]:
# labeled_auto[0]
# viewer = ImageViewer(labeled_auto[0])
# viewer.show()

[]

In [16]:
height, width, layers = labeled_auto[0].shape

fps = 60
video = cv2.VideoWriter('auto_hough_movement.mov', cv2.VideoWriter_fourcc('m', 'p', '4', 'v'), fps, (width,height))
# video = cv2.VideoWriter('auto_hough_movement.mov', cv2.VideoWriter_fourcc(*'XVID'), fps, (width,height))
for i in range(len(labeled_auto)):    
    video.write(labeled_auto[i])
cv2.destroyAllWindows()
video.release()

In [11]:
for filename in glob.glob("circle*"):
    os.remove(filename) 